This notebook takes the raw tracks for each observation and finds all crossing tracks and extracts all nessisary assosiated information for population estimates including things like bat size, frame darkness etc. All this information is saved in one dictionary for each observation which can be used in other notebooks for further processing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

from bat_functions import mark_bats_on_image, draw_tracks_on_frame
from bat_functions import get_bat_accumulation, piecewise_linear
from bat_functions import threshold_short_tracks, calculate_height
from bat_functions import calculate_bat_multiplier, combined_bat_multiplier
from bat_functions import measure_crossing_bats
from CountLine import CountLine
import cv2
import matplotlib as mpl
import utm
import matplotlib.pyplot as plt
# import rasterio

In [ ]:
shift = 0 # loss on each side from not padding during detection

HFOV = 85.8 # degrees
HCONST = 1454.9 # pixels
WCONST = 1453.7 # pixels
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 

image_file = '.../bats/saved_frames/Musole_Tower/GP029057/GP029057_new03-0000-circle-alt-thresh.jpg'
image_file = '.../kasanka-bats/processed/deep-learning/19Nov/BBC/example-frames/19Nov_BBC_obs-ind_40500.jpg'
frame_height = plt.imread(image_file).shape[0]
frame_width = plt.imread(image_file).shape[1]

In [ ]:
# camera_distances = {'Chyniangale': 306.009,
#                     'Chinyingale': 306.009,
#                     'Puku': 383,
#                     'FibweParking': 1002,
#                     'FibweParking2':1003,
#                     'Sunset': 448,
#                     'NotChyniangale': 266,
#                     'NotChipingale': 266,
#                     'NotChinyingale': 266,
#                     'Musole_Parking': 408,
#                     'MusolaParking': 408,
#                     'Musole_path': 185,
#                     'MusolaPath': 286,
#                     'Musole_Path2': 286,
#                     'FibwePublic': 1024,
#                     'Fibwe_Public': 1024,
                    
#                     'MusoleTower': 256,
#                     'MusolaTower': 256,
#                     'BBC': 521}
camera_distances = {'Chyniangale': 306.009,
                    'Puku': 383,
                    'FibweParking': 1002,
                    'FibweParking2':1003,
                    'Sunset': 448,
                    'NotChyniangale': 266,
                    'MusoleParking': 408,
                    'MusolePath': 185,
                    'MusolePath2': 286,
                    'FibwePublic': 1024,
                    'MusoleTower': 256,
                    'BBC': 521}

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"

## Show demo image

In [ ]:
root_frame_folder = ".../Elements/bats"
date = "17Nov"
observation_name = "BBC"

frame_files = sorted(
    glob.glob(os.path.join(root_frame_folder,
                           date,
                           observation_name, 
                           "*/*.jpg"
                    )
    )
)
positions = np.load(os.path.join(root_folder, 
                                 date, observation_name, 
                                 'centers.npy'), 
                    allow_pickle=True)
print(len(frame_files), len(positions))

In [ ]:
frame_ind = 20000
plt.figure(figsize=(20,20))
im = plt.imread(frame_files[frame_ind])
plt.imshow(im / im.max())
plt.scatter(shift + positions[frame_ind][:,0], 
            shift + positions[frame_ind][:,1], 
            s=1, c='r')

## Find crossing tracks

In [ ]:
with_multiplier= True
should_save = False
num_cameras = None

calc_crossing = True

if calc_crossing:
    folders = glob.glob(
        '.../kasanka-bats/processed/deep-learning/*Nov'
    )
    day_folders = sorted(folders)
        
    observations = {}
    for day_folder in day_folders[:1]:
        print(day_folder)
        
        date = os.path.basename(day_folder)
        track_files = sorted(
            glob.glob(os.path.join(day_folder, '*/raw_tracks.npy'))
        )
        observations[date] = {}
        for track_file in track_files[:num_cameras]:
            crossing_track_file = os.path.join(os.path.dirname(track_file), 
                                           'crossing_tracks.npy')
            if os.path.exists(crossing_track_file):
                continue
            try:
                raw_track_list = np.load(track_file, allow_pickle=True)
                tracks_list = threshold_short_tracks(raw_track_list, min_length_threshold=2)
                crossing_tracks_list = measure_crossing_bats(
                    tracks_list, frame_height=frame_height,
                    frame_width=frame_width, count_across=True, 
                    count_out=True, num_frames=None, with_rects=with_multiplier
                )
                if should_save:
                    np.save(crossing_track_file, 
                            np.array(crossing_tracks_list, dtype=object)
                           )
            except:
                print('failed')



In [ ]:
day_folders = sorted(glob.glob('.../kasanka-bats/processed/deep-learning/*Nov'))
observations = {}
for day_folder in day_folders[:]:
    print(day_folder)

    date = os.path.basename(day_folder)
    track_files = sorted(glob.glob(os.path.join(day_folder, '*/crossing_tracks.npy')))
    observations[date] = {}
    
    for track_file in track_files: 
        camera = track_file.split('/')[-2]
        obs = {'date': date,
               'camera': camera,
               }
        crossing_tracks_list = np.load(track_file, allow_pickle=True)
        darkness_means = np.load(os.path.join(day_folder, camera, 'blue-means.npy'))
        passing_bat_frame = []
        passing_bat_size = []
        passing_bat_track_ind = []
        passing_bat_direction = []
        passing_bat_track_id = []
        passing_bat_darkness = []
        passing_track_length = []
        for track_ind, track in enumerate(crossing_tracks_list):
            if track['crossed'] > 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(1)
                passing_bat_darkness.append(darkness_means[track['crossed']])
                passing_track_length.append(len(track['track']))

            elif track['crossed'] < 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(-1)
                passing_bat_darkness.append(darkness_means[-track['crossed']])
                passing_track_length.append(len(track['track']))

        obs['frames'] = np.array(passing_bat_frame)
        obs['mean_wing'] = np.array(passing_bat_size)
        obs['ids'] = np.array(passing_bat_track_id)
        obs['direction'] = np.array(passing_bat_direction)
        obs['darkness'] = np.array(passing_bat_darkness)
        obs['track_length'] = np.array(passing_track_length)
        observations[date][camera] = obs

day_folders = sorted(glob.glob('.../kasanka-bats/processed/deep-learning/*Nov'))
observations = {}
for day_folder in day_folders[:]:
    print(day_folder)

    date = os.path.basename(day_folder)
    track_files = sorted(glob.glob(os.path.join(day_folder, '*/crossing_tracks.npy')))
    observations[date] = {}
    
    for track_file in track_files: 
        camera = track_file.split('/')[-2]
        obs = {'date': date,
               'camera': camera,
               }
        crossing_tracks_list = np.load(track_file, allow_pickle=True)
        darkness_means = np.load(os.path.join(day_folder, camera, 'blue-means.npy'))
        passing_bat_frame = []
        passing_bat_size = []
        passing_bat_track_ind = []
        passing_bat_direction = []
        passing_bat_track_id = []
        passing_bat_darkness = []
        for track_ind, track in enumerate(crossing_tracks_list):
            if track['crossed'] > 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(1)
                passing_bat_darkness.append(darkness_means[track['crossed']])

            elif track['crossed'] < 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(-1)
                passing_bat_darkness.append(darkness_means[-track['crossed']])

        obs['frames'] = np.array(passing_bat_frame)
        obs['mean_wing'] = np.array(passing_bat_size)
        obs['ids'] = np.array(passing_bat_track_id)
        obs['direction'] = np.array(passing_bat_direction)
        obs['darkness'] = np.array(passing_bat_darkness)
        observations[date][camera] = obs

In [ ]:
obs.keys()

In [ ]:
observation_root = '.../kasanka-bats/processed/deep-learning/observations'
for date, day_obs in observations.items():
    day_folder = os.path.join(observation_root, date)
    os.makedirs(day_folder, exist_ok=True)
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        obs_name = f'{date}-observation-{cam_name}.npy'
        obs_file = os.path.join(day_folder, obs_name)
        np.save(obs_file, obs)

In [ ]:
camera_locations = {'FibweParking2': [-12.5903393, 30.2525047],	
                    'FibweParking': [-12.5903393, 30.2525047],
                    'Chyniangale': [-12.5851284, 30.245529],	
                    'BBC': [-12.5863538, 30.2484985],
                    'Sunset': [-12.585784, 30.240003],
                    'NotChyniangale': [-12.5849206,	30.2436135],
                    'MusoleParking': [-12.58787, 30.2401],	
                    'MusolePath2': [-12.589544,	30.242488],	
                    'MusolePath': [-12.589544,	30.242488],
                    'Puku': [-12.584838, 30.24137],	
                    'FibwePublic': [-12.592537, 30.2515924],	
                    'MusoleTower': [-12.589434, 30.244736],
                    }

forest_border = [[-12.585957, 30.242762],
                 [-12.586763, 30.246229],
                 [-12.589854, 30.250597],
                 [-12.591381, 30.249095],
                 [-12.589182, 30.245566],
                 [-12.587557, 30.241598]
                ]

center = [-12.587444, 30.244290]

In [ ]:
# def get_camera_angles(camera_utms, center_utm):
#     camera_angles = {}
#     for camera, camera_utm in camera_utms.items():
#         dif = camera_utm - center_utm
#         camera_angles[camera] = np.arctan2(dif[1], dif[0])
#     return camera_angles

# def get_camera_distances(camera_utms, center_utm):
#     camera_distances = {}
#     for camera, camera_utm in camera_utms.items():
#         diff = camera_utm - center_utm
#         camera_distances[camera] = np.sum(np.sqrt(diff ** 2))
#     return camera_distances

# def get_camera_borders(camera_utms, camera_angles):
#     camera_border = {}
#     for camera, camera_utm in camera_utms.items():
#         min_neg = -10000
#         min_pos = 100000
#         max_pos = 0 # for border case where focal is positive angle and closest cclock is negative
#         all_pos = True # for same case a last comment
#         max_neg = 0 # for border case where focal is positive angle and closest cclock is negative
#         all_neg = True # for same case a last comment
#         max_camera = None
#         camera_border[camera] = {'cclock': None,
#                                  'cclock_angle': None,
#                                  'clock': None,
#                                  'clock_angle': None
#                                 }
#         for alt_camera, alt_camera_utm in camera_utms.items():
            
#             if camera == alt_camera:
#                 continue

#             dif = camera_angles[camera] - camera_angles[alt_camera]
#             if dif < 0:
#                 all_pos = False
#                 if dif > min_neg:
#                     min_neg = dif
#                     camera_border[camera]['cclock'] = alt_camera
#                     camera_border[camera]['cclock_angle'] = dif / 2
#                 if dif < max_neg:
#                     max_neg = dif 
#                     max_camera = alt_camera

#             if dif > 0:
#                 all_neg = False
#                 if dif < min_pos:
#                     min_pos = dif
#                     camera_border[camera]['clock'] = alt_camera
#                     camera_border[camera]['clock_angle'] = dif / 2
#                 if dif > max_pos:
#                     max_pos = dif 
#                     max_camera = alt_camera

#         if all_pos:
#             camera_border[camera]['cclock'] = max_camera
#             camera_border[camera]['cclock_angle'] = (max_pos - 2*np.pi) / 2
#         if all_neg:
#             camera_border[camera]['clock'] = max_camera
#             camera_border[camera]['clock_angle'] = (max_neg + 2*np.pi) / 2
            
#     return camera_border

In [ ]:
all_camera_utms = latlong_dict_to_utm(camera_locations)
forest_utms = []
for f_latlon in forest_border:
    f_utm = utm.from_latlon(*f_latlon)
    forest_utms.append([f_utm[0], f_utm[1]])
forest_utms = np.array(forest_utms)
    
# center_utm = utm.from_latlon(*center)
# center_utm = np.array([center_utm[0], center_utm[1]])

In [ ]:
# angles = get_camera_angles(all_camera_utms, center_utm)

In [ ]:
camera_utm_array = []
for utm in all_camera_utms.values():
    camera_utm_array.append(utm)
camera_utm_array = np.array(camera_utm_array)
max_area_x = (np.max(camera_utm_array[:,0]) - np.min(camera_utm_array[:,0])).astype(int)
max_area_y = (np.max(camera_utm_array[:,1]) - np.min(camera_utm_array[:,1])).astype(int)
area = np.zeros((max_area_y, max_area_x), dtype=np.uint8)

In [ ]:
norm_forest = np.copy(forest_utms)
area_x_origin = np.min(camera_utm_array[:, 0])
area_y_origin = np.min(camera_utm_array[:, 1])
norm_forest[:, 0] = forest_utms[:, 0] - area_x_origin
norm_forest[:, 1] = forest_utms[:, 1] - area_y_origin
area = cv2.drawContours(area, [norm_forest.astype(np.int32)], -1, 255, -1)

In [ ]:
plt.imshow(area)

In [ ]:
map_file = '.../bats-data/maps/kasanka-utm.tiff'
forest_map_dataset = rasterio.open(map_file)

In [ ]:
forest_map_dataset.transform
forest_map_dataset.count

In [ ]:
forest_map = [forest_map_dataset.read(band_ind) for band_ind in range(1, 4)]
forest_map = np.array(forest_map)
forest_map = forest_map.transpose(1, 2, 0)

In [ ]:
plt.imshow(forest_map)

In [ ]:
width = np.abs(forest_map_dataset.bounds.left - forest_map_dataset.bounds.right).astype(int)
height = np.abs(forest_map_dataset.bounds.top - forest_map_dataset.bounds.bottom).astype(int)
area = np.zeros((height, width), dtype=np.uint8)

In [ ]:
norm_forest = np.copy(forest_utms)
area_x_origin = forest_map_dataset.bounds.left
area_y_origin = forest_map_dataset.bounds.bottom
norm_forest[:, 0] = forest_utms[:, 0] - area_x_origin
norm_forest[:, 1] = forest_utms[:, 1] - area_y_origin
area = cv2.drawContours(area, [norm_forest.astype(np.int32)], -1, 255, -1)

In [ ]:
forest_map_dataset.bounds

In [ ]:
plt.imshow(area)

In [ ]:
plt.imshow(forest_map)
for point in forest_utms:
    row, col = forest_map_dataset.index(point[0], point[1])
    print(point[1], point[0], row, col)
    plt.scatter(col, row)

In [ ]:
# def get_camera_locations(observations, all_camera_locations, exclude=False):
#     """Return dict of all camera locations used in observation"""
#     camera_locations = {}
#     for camera, obs in observations.items():
#         if exclude:
#             if 'exclude' in obs.keys():
#                 if obs['exclude']:
#                     continue
#         camera_locations[obs['camera']] = all_camera_locations[obs['camera']]
#     return camera_locations
    


# def get_day_total(observations, center_utm, all_camera_utms, 
#                   frame_width, wingspan, exclude=False, 
#                   correct_darkness=False, wing_scale=1):
#     """ exlude: to manually remove certain cameras
#         correct_darkness: divide by accuracy estimated for given darkness"""

#     parameters = [1.57454778e+01, 9.37398964e-01, 7.18914388e-02, -1.27575036e-04]
    
#     frac_sum = 0
#     total = 0
#     obs_totals = []
    
#     camera_utms = get_camera_locations(observations, all_camera_utms, exclude=True)
#     for obs in observations.values():
#         if exclude:
#             if 'exclude' in obs.keys():
#                 if obs['exclude']:
#                     continue
#         camera_distances = get_camera_distances(camera_utms, center_utm)
#         obs['multiplier'] = combined_bat_multiplier(frame_width, 
#                                                     wingspan, 
#                                                     obs['mean_wing']*wing_scale, 
#                                                     camera_distances[obs['camera']]
#                                                    )
#         if correct_darkness:
#             acc = piecewise_linear(
#                 obs['darkness'], *parameters)
#             obs['total_darkness'] = np.sum(obs['multiplier'] * obs['direction'] * (1/acc))
#         obs['total'] = np.sum(obs['multiplier'] * obs['direction'])
#         obs['total_unscaled'] = np.sum(obs['direction'])
#         camera_angles = get_camera_angles(camera_utms, center_utm)
#         camera_border = get_camera_borders(camera_utms, camera_angles)
#         angle = (-camera_border[obs['camera']]['cclock_angle'] 
#                  + camera_border[obs['camera']]['clock_angle']
#                 )
#         frac = angle / (np.pi * 2)
#         obs['fraction_total'] = frac
#         frac_sum += frac
#         if correct_darkness:
#             total += obs['total_darkness'] * frac
#             obs_totals.append(obs['total_darkness'])
#         else:
#             total += obs['total'] * frac
#             obs_totals.append(obs['total'])

#     if len(obs_totals) > 0:
#         mean_total = np.mean(obs_totals)
#     else:
#         mean_total = 0

#     return total, mean_total
    

In [ ]:
obs['multiplier'][:10]

In [ ]:
parameters = [1.57454778e+01, 9.37398964e-01, 7.18914388e-02, -1.27575036e-04]
parameters_alt = [ 2.53107930e+01,  9.59547293e-01,  2.70747111e-02, -1.18602475e-03]
parameters_alt2 = [1.03891791e+01, 8.78179573e-01, 1.86387502e-01, 1.77968688e-04]
should_save=False
show_alt_params = True
save_folder = os.path.join(plot_folder, 'bat-accumulation')

fontsize = 15

num_cols = 5

max_bats = 0
# so all plots have same scale
for date, day_obs in observations.items():
    get_day_total(day_obs, center_utm['middle'], 
                  all_camera_utms, FRAME_WIDTH, WINGSPAN, 
                  exclude=exclude, correct_darkness=True
                 )                                         
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters_alt)
        if bat_accumulation[-1] > max_bats:
            max_bats = bat_accumulation[-1]
max_bats += 1000  


for date, day_obs in observations.items():
    fig, axs = plt.subplots(2, num_cols, figsize=(20,20))
    
    total = 0
    total0 = 0
    total1 = 0
    total2 = 0
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        darkness_means = np.load(
            os.path.join(root_folder, date, cam_name, 'blue-means.npy')
        )
        c0 = 'b'
        bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters)
        axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='corrected', c=c0)
        total0 += bat_accumulation[-1]
        bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters, w_darkness=False)
        axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='raw', c='k')
        total += bat_accumulation[-1]
        beginning_error_frame = np.argmax(darkness_means < parameters[0])
        axs[cam_ind//num_cols, cam_ind%num_cols].axvline(beginning_error_frame, c=c0)
        if show_alt_params:
            c1 = 'r'
            bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters_alt)
            axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='alt params', c=c1)
            beginning_error_frame = np.argmax(darkness_means < parameters_alt[0])
            axs[cam_ind//num_cols, cam_ind%num_cols].axvline(beginning_error_frame, c=c1)
            total1 += bat_accumulation[-1]
            
            c2 = 'g'
            bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters_alt2)
            axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='alt params', c=c2)
            beginning_error_frame = np.argmax(darkness_means < parameters_alt2[0])
            axs[cam_ind//num_cols, cam_ind%num_cols].axvline(beginning_error_frame, c=c2)
            total2 += bat_accumulation[-1]
        axs[cam_ind//num_cols, cam_ind%num_cols].set_title(cam_name, fontsize=fontsize*1.5)
    for ax_ind, ax in enumerate(axs.reshape(-1)):
        ax.set_ylim(top=max_bats)
        ax.set_xlabel('frame number', fontsize=fontsize)
        ax.tick_params(axis='y', labelsize=fontsize)
        ax.yaxis.set_major_formatter(
            mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        if ax_ind % num_cols != 0:
            ax.tick_params(labelleft=False)  
    for r in range(len(axs)):
        axs[r, 0].set_ylabel('number of bats seen', fontsize=fontsize)
    fig.suptitle(f'{date} total bats: \n raw: {total:,.0f}, blue: {total0:,.0f}, red: {total1:,.0f}, green: {total2:,.0f}', size=fontsize*2)
    if show_alt_params:
        plot_name = 'bat-accumulation-scaled-break-{}-comparison-{}.png'.format(parameters[0], date)
    else:
        plot_name = 'bat-accumulation-scaled-break-{}-{}.png'.format(parameters[0], date)
    plot_file = os.path.join(save_folder, plot_name)
    if should_save:
        fig.savefig(plot_file, bbox_inches='tight')



# day = '16Nov'

# for cam_ind, (cam_name, obs) in enumerate(observations['16Nov'].items()):
    
#     darkness_means = np.load(os.path.join(root_folder, day, cam_name, 'blue-means.npy'))
#     beginning_error_frame = np.argmax(darkness_means < parameters[0])
#     plt.figure()
#     plt.axvline(beginning_error_frame)
#     bat_accumulation = get_bat_accumulation(obs['frames'], obs)
#     plt.plot(bat_accumulation, label='estimate')
#     bat_accumulation = get_bat_accumulation(obs['frames'], obs, w_darkness=False)
#     plt.plot(bat_accumulation, label='without error scaling')
#     plt.legend()
#     plt.title(cam_name)
        


In [ ]:
plot_folder = '.../bats-data/plots'
plt.style.use('default')

In [ ]:
save_folder = os.path.join(plot_folder, 'bat-accumulation')
os.makedirs(save_folder, exist_ok=True)
should_save = False
fontsize = 15

num_cols = 5

max_bats = 0
# so all plots have same scale
for date, day_obs in observations.items():
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        bat_accumulation = get_bat_accumulation(obs['frames'])
        if bat_accumulation[-1] > max_bats:
            max_bats = bat_accumulation[-1]
max_bats += 1000           
            
for date, day_obs in observations.items():
    fig, axs = plt.subplots(2, num_cols, figsize=(20,20))
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        bat_accumulation = get_bat_accumulation(obs['frames'])
        axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation)
        axs[cam_ind//num_cols, cam_ind%num_cols].set_title(cam_name, fontsize=fontsize*1.5)
    for ax_ind, ax in enumerate(axs.reshape(-1)):
        ax.set_ylim(top=max_bats)
        ax.set_xlabel('frame number', fontsize=fontsize)
        ax.tick_params(axis='y', labelsize=fontsize)
        ax.yaxis.set_major_formatter(
            mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        if ax_ind % num_cols != 0:
            ax.tick_params(labelleft=False)  
    for r in range(len(axs)):
        axs[r, 0].set_ylabel('number of bats seen', fontsize=fontsize)
    fig.suptitle(date, size=fontsize*3)
    plot_name = 'bat-accumulation-{}.png'.format(date)
    plot_file = os.path.join(save_folder, plot_name)
    if should_save:
        fig.savefig(plot_file, bbox_inches='tight')

In [ ]:
parameters = [1.57454778e+01, 9.37398964e-01, 7.18914388e-02, -1.27575036e-04]
# parameters_alt = [ 2.53107930e+01,  9.59547293e-01,  2.70747111e-02, -1.18602475e-03]
# parameters_alt2 = [1.03891791e+01, 8.78179573e-01, 1.86387502e-01, 1.77968688e-04]
should_save = True
show_alt_params = False
save_folder = os.path.join(plot_folder, 'bat-accumulation')

fontsize = 15

num_cols = 5

max_bats = 0
# so all plots have same scale
for date, day_obs in observations.items():
    get_day_total(day_obs, center_utm['middle'], 
                  all_camera_utms, FRAME_WIDTH, WINGSPAN, 
                  exclude=exclude, correct_darkness=True,
                  wing_scale=1.15

                 )                                         
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters_alt)
        if bat_accumulation[-1] > max_bats:
            max_bats = bat_accumulation[-1]
max_bats += 1000  


for date, day_obs in observations.items():
    fig, axs = plt.subplots(2, num_cols, figsize=(20,20))
    
    wing_scales = [0.85, 1.0, 1.15]
    totals = [0 for _ in wing_scales]
    colors = ['r', 'b', 'r']
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        
        for scale_ind, wing_scale in enumerate(wing_scales):
            
            camera_distances = get_camera_distances(all_camera_utms, center_utm['middle'])
            
#             print(scale_ind, cam_ind)
            obs['multiplier'] = combined_bat_multiplier(FRAME_WIDTH, 
                                                            WINGSPAN, 
                                                            obs['mean_wing']*wing_scale, 
                                                            camera_distances[obs['camera']]
                                                           )
            bat_accumulation = get_bat_accumulation(obs['frames'], obs, parameters)
            axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, c=colors[scale_ind])
            totals[scale_ind] += bat_accumulation[-1]
        axs[cam_ind//num_cols, cam_ind%num_cols].set_title(cam_name, fontsize=fontsize*1.5)
    for ax_ind, ax in enumerate(axs.reshape(-1)):
        ax.set_ylim(top=max_bats)
        ax.set_xlabel('frame number', fontsize=fontsize)
        ax.tick_params(axis='y', labelsize=fontsize)
        ax.yaxis.set_major_formatter(
            mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        if ax_ind % num_cols != 0:
            ax.tick_params(labelleft=False)  
    for r in range(len(axs)):
        axs[r, 0].set_ylabel('number of bats seen', fontsize=fontsize)
    fig.suptitle(f'{date} wing error: \n total bats .85: {totals[0]:,.0f}, 1.0: {totals[1]:,.0f}, 1.15: {totals[2]:,.0f}', size=fontsize*2)

    plot_name = 'bat-accumulation-wing-error-{}.png'.format(date)
    plot_file = os.path.join(save_folder, plot_name)
    if should_save:
        fig.savefig(plot_file, bbox_inches='tight')

In [ ]:
np.mean(obs['mean_wing']*1.0)

In [ ]:
# Manually exclude cameras that had issues
observations['18Nov']['MusolePath']['exclude'] = True
observations['20Nov']['MusolePath']['exclude'] = True

In [ ]:
exclude = True
scale = 10
total_mean_maps = []
total_weighted_maps = []
for _ in range(len(observations)):
    total_mean_maps.append(np.zeros(((int(area.shape[0]/scale)+1, int(area.shape[1]/scale)+1))))
    total_weighted_maps.append(np.zeros(((int(area.shape[0]/scale)+1, int(area.shape[1]/scale)+1))))
                      
for x_ind, x in enumerate(range(0, area.shape[1], scale)):
    for y_ind, y in enumerate(range(0, area.shape[0], scale)):
        if area[y, x] == 0:
            continue
        x_utm = x + area_x_origin
        y_utm = y + area_y_origin
        center_utm = np.array([x_utm, y_utm])
        for day_ind, day in enumerate(observations.values()):
            day_total, day_total_mean = get_day_total(day, center_utm, all_camera_utms, 
                                                      FRAME_WIDTH, WINGSPAN, exclude=exclude)
            
            total_mean_maps[day_ind][y_ind, x_ind] = day_total_mean
            total_weighted_maps[day_ind][y_ind, x_ind] = day_total
                         
                
for map_ind, total_map in enumerate(total_mean_maps):
    total_mean_maps[map_ind] = cv2.resize(total_map, (area.shape[1], area.shape[0]), 
                                          interpolation=cv2.INTER_NEAREST)
for map_ind, total_map in enumerate(total_weighted_maps):
    total_weighted_maps[map_ind] = cv2.resize(total_map, (area.shape[1], area.shape[0]), 
                                          interpolation=cv2.INTER_NEAREST)      

In [ ]:
max_count = 0
for mean_map, weighted_map in zip(total_mean_maps, total_weighted_maps):
    day_max = np.max([np.max(mean_map), np.max(weighted_map)])
    if day_max > max_count:
        max_count = day_max
print(max_count)

In [ ]:
weighted.shape

In [ ]:
# area_x_origin = forest_map_dataset.bounds.left 
# area_y_origin = forest_map_dataset.bounds.top

# total_weighted_maps = [np.copy(forest_map), np.copy(forest_map)]
# total_mean_maps = [np.copy(forest_map), np.copy(forest_map)]

In [ ]:
# im = np.ones((20,20,3), dtype=np.uint8)
# print(type(im[0,0,0]), im.shape)
# im = total_mean_maps[0]
# # cv2.circle(im, 
# #            (int(utm[0] - area_x_origin), int(utm[1] - area_y_origin)), 
# #            15, (255,255,255), -1)
# print(type(im[0,0,0]), im.shape)

In [ ]:
plt.imshow(forest_map)
mean_maps = cv2.resize(total_mean_maps[0], (forest_map.shape[1], forest_map.shape[0]))
alpha_mask = np.where(mean_maps>0, 1.0, 0.0)
plt.imshow(mean_maps[::-1], alpha=alpha_mask[::-1]*.4)

In [ ]:
mean_maps.shape

In [ ]:
import matplotlib.cm as cm
import matplotlib

alpha=1.0

save_folder = os.path.join(plot_folder, 'various-centers')
os.makedirs(save_folder, exist_ok=True)

cmap = cm.seismic
for weighted, mean, (date, day_obs) in zip(total_weighted_maps, total_mean_maps, observations.items()):
    fig, axs = plt.subplots(1, 2, figsize=(20,7))
    camera_utms = get_camera_locations(day_obs, all_camera_utms, exclude=exclude)
    print(camera_utms.keys())
    for camera, utm in camera_utms.items():
        cv2.circle(weighted, (int(utm[0] - area_x_origin), int(utm[1] - area_y_origin)), 15, 800000, -1)
        cv2.circle(mean, (int(utm[0] - area_x_origin), int(utm[1] - area_y_origin)), 15, 800000, -1)
    axs[0].imshow(forest_map)
    weighted = cv2.resize(weighted, (forest_map.shape[1], forest_map.shape[0]))
    alpha_mask = np.where(weighted>0, 1.0, 0.0) * alpha
    n = axs[0].imshow(weighted[::-1], cmap=cmap, vmin=0, vmax=max_count, alpha=alpha_mask[::-1])
    axs[0].set_aspect('equal', adjustable='box')
    axs[0].set_xticks([]) 
    axs[0].set_yticks([]) 
    axs[0].set_title('Weighted Cameras', fontsize=15)
    axs[1].set_aspect('equal', adjustable='box')
    axs[1].imshow(forest_map)
    mean = cv2.resize(mean, (forest_map.shape[1], forest_map.shape[0]))
    alpha_mask = np.where(mean>0, 1.0, 0.0) * alpha
    m = axs[1].imshow(mean[::-1], cmap=cmap, vmin=0, vmax=max_count, alpha=alpha_mask[::-1])
    axs[1].set_xticks([]) 
    axs[1].set_yticks([]) 
    axs[1].set_title('Straight Average', fontsize=15)
    norm = matplotlib.colors.Normalize(vmin=0, vmax=max_count)
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)
    cbar = fig.colorbar(sm)
    cbar.set_label('total bats', rotation=270, labelpad=15, fontsize=15)
#     l, y=1.05, rotation=0
    title = 'total-bats-from-various-centers-map-{}'.format(date)
    fig.suptitle(title.replace('-', ' '), size=30)
    if should_save:
        fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')

In [ ]:
center_utm = {'middle': np.array([200450, 8606950]),
              'right': np.array([200800, 8606900])}

In [ ]:
center = 'middle'

save_folder = os.path.join(plot_folder, 'camera-weighting-diagram')
os.makedirs(save_folder, exist_ok=True)

def rotate(vec, angle):
    new_x = vec[0] * np.cos(angle) - vec[1] * np.sin(angle)
    new_y = vec[0] * np.sin(angle) + vec[1] * np.cos(angle)
    return np.array([new_x, new_y])

import matplotlib.cm as cm

for center in ['middle', 'right']:
    if center == 'middle':
        color = 'r'
    elif center == 'right':
        color = 'r'
    total_angle = 0
    
    fig, ax = plt.subplots(figsize=(20,20))
    ax.imshow(forest_map)

    camera_utms = get_camera_locations(observations['17Nov'], all_camera_utms, exclude=True)
    camera_border = get_camera_borders(camera_utms, center_utm[center])
    
    for camera, camera_utm in camera_utms.items():
    #     c = cm.viridis((camera_angles[camera] + np.pi)/ (2*np.pi))
        camera_plot = forest_map_dataset.index(*camera_utm)
        camera_plot = [camera_plot[1], camera_plot[0]]
        ax.scatter(*camera_plot, c='w', s=100)
        cclock = camera_border[camera]['cclock']
        clock = camera_border[camera]['clock']
        ax.annotate(camera, camera_plot, fontsize=15, 
                    xytext=[camera_plot[0], camera_plot[1]-40], 
                    ha='center', va='top', color='white', 
                    bbox=dict(boxstyle = "square",
                      facecolor = "gray", alpha=.7))

        mid = rotate(np.array(camera_utms[camera])-np.array(center_utm[center]), 
                     -camera_border[camera]['clock_angle'])
        mid = mid + np.array(center_utm[center])
        mid_plot = forest_map_dataset.index(*mid)
        mid_plot = [mid_plot[1], mid_plot[0]]
        center_plot = forest_map_dataset.index(*center_utm[center])
        center_plot = [center_plot[1], center_plot[0]]
        ax.plot([mid_plot[0], center_plot[0]], [mid_plot[1], center_plot[1]], c=color)

        mid = rotate(np.array(camera_utms[camera])-np.array(center_utm[center]), 
                     -camera_border[camera]['cclock_angle'])
        mid = mid + np.array(center_utm[center])
        mid_plot = forest_map_dataset.index(*mid)
        mid_plot = [mid_plot[1], mid_plot[0]]
        ax.plot([mid_plot[0], center_plot[0]], [mid_plot[1], center_plot[1]], c=color)
    #     ax.scatter(*(mid + np.array(center_utm)), c='r', s=50)
        ax.set_aspect('equal', adjustable='box')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    #     ax.set_xlabel('Meters (UTM)', fontsize=15)
    #     ax.set_ylabel('Meters (UTM)', fontsize=15)

        total_angle += -camera_border[camera]['cclock_angle']
        total_angle += camera_border[camera]['clock_angle']
    print(total_angle / np.pi)
    ax.scatter(*center_plot, s=100, c='r')
    ax.set_aspect('equal', adjustable='box')
    title = 'camera-weighting-center-{}'.format(center)
    ax.set_title(title.replace('-', ' '), size=30)
    if should_save:
        fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')

In [ ]:
center = 'middle'
for day_ind, (date, day) in enumerate(observations.items()):
    if not day:
        continue
        
    print('--')
    for cd in [True, False]:
        day_total, day_total_mean = get_day_total(day, center_utm[center], all_camera_utms, 
                                                  FRAME_WIDTH, WINGSPAN, exclude=exclude,
                                                  correct_darkness=cd
                                                 )
        print(day_total)
        

In [ ]:
for center in ['middle', 'right']:

    save_folder = os.path.join(plot_folder, 'center-summary-figure')
    os.makedirs(save_folder, exist_ok=True)

    fig, axs = plt.subplots(len(observations), 3, figsize=(20, 6*len(observations)), 
                            subplot_kw=dict(polar=True))

    max_bats_per_degree = 0
    max_bats_per_camera = 0

    for day_ind, (date, day) in enumerate(observations.items()):
        if not day:
            continue
        day_total, day_total_mean = get_day_total(day, center_utm[center], all_camera_utms, 
                                                  FRAME_WIDTH, WINGSPAN, exclude=exclude,
                                                  correct_darkness=True
                                                 )
        fractions = []
        cameras = []
        totals = []
        contribution = []
        angles = []

        camera_angles = get_camera_angles(all_camera_utms, center_utm[center])
        for camera, obs in day.items():
            if exclude:
                if 'exclude' in obs.keys():
                    continue
            fractions.append(obs['fraction_total'])
            cameras.append(camera)
            totals.append(obs['total_darkness']/360)
            contribution.append(obs['total_darkness'] * obs['fraction_total'])
            angles.append(camera_angles[camera])

        max_bats_per_degree_day = np.max(totals)
        if max_bats_per_degree_day > max_bats_per_degree:
            max_bats_per_degree = max_bats_per_degree_day
        max_bats_per_camera_day = np.max(contribution)
        if max_bats_per_camera_day > max_bats_per_camera:
            max_bats_per_camera = max_bats_per_camera_day

        # fractions = [f for _,f in sorted(zip(angles,fractions))]
        frac_ind = 2
        axs[day_ind, frac_ind].set_thetalim(-np.pi, np.pi)
        axs[day_ind, frac_ind].scatter(angles, fractions, s=30)
        axs[day_ind, frac_ind].set_xticks(angles)
        _ = axs[day_ind, frac_ind].set_xticklabels(cameras)
        axs[day_ind, frac_ind].set_title('Fraction', size='x-large')

        total_ind = 1
        axs[day_ind, total_ind].set_thetalim(-np.pi, np.pi)
        axs[day_ind, total_ind].scatter(angles, totals, s=30)
        axs[day_ind, total_ind].set_xticks(angles)
        _ = axs[day_ind, total_ind].set_xticklabels(cameras)
        axs[day_ind, total_ind].set_title('Bats per degree', size='x-large')

        contrib_ind = 0
        axs[day_ind, contrib_ind].set_thetalim(-np.pi, np.pi)
        axs[day_ind, contrib_ind].scatter(angles, contribution, s=30)
        axs[day_ind, contrib_ind].set_xticks(angles)
        _ = axs[day_ind, contrib_ind].set_xticklabels(cameras)
        axs[day_ind, contrib_ind].set_title('Bats from camera', size='x-large')

        pad = 60
        left_label = f'{date} \n {int(np.sum(contribution)):,} bats'
        axs[day_ind,0].annotate(left_label, xy=(0, 0.5), 
                                xytext=(-axs[day_ind,0].yaxis.labelpad - pad, 0),
                                xycoords=axs[day_ind,0].yaxis.label, 
                                textcoords='offset points',
                                size='xx-large', ha='right', va='center'
                               )
    for day_ind, (date, day) in enumerate(observations.items()):
        axs[day_ind, contrib_ind].set_rlim(0, max_bats_per_camera)
        axs[day_ind, total_ind].set_rlim(0, max_bats_per_degree)

    title = f'summary-for-center-{center}'
    fig.suptitle(title.replace('-', ' '), size=30, va='center')
    if should_save:
        fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')

In [ ]:
axis_labels_day_ind = 0

for center in ['middle', 'right'][:]:

    save_folder = os.path.join(plot_folder, 'center-summary-figure')
    os.makedirs(save_folder, exist_ok=True)

    fig, axs = plt.subplots(1, 3, figsize=(20, 6), 
                            subplot_kw=dict(polar=True))

    max_bats_per_degree = 0
    max_bats_per_camera = 0

    for day_ind, (date, day) in enumerate(observations.items()):
        if not day:
            continue
        day_total, day_total_mean = get_day_total(day, center_utm[center], all_camera_utms, 
                                                  FRAME_WIDTH, WINGSPAN, exclude=exclude)
        fractions = []
        cameras = []
        totals = []
        contribution = []
        angles = []
        
        camera_angles = get_camera_angles(all_camera_utms, center_utm[center])
        for camera, obs in day.items():
            if exclude:
                if 'exclude' in obs.keys():
                    continue
            fractions.append(obs['fraction_total'])
            cameras.append(camera)
            totals.append(obs['total']/360)
            contribution.append(obs['total'] * obs['fraction_total'])
            angles.append(camera_angles[camera])

        max_bats_per_degree_day = np.max(totals)
        if max_bats_per_degree_day > max_bats_per_degree:
            max_bats_per_degree = max_bats_per_degree_day
        max_bats_per_camera_day = np.max(contribution)
        if max_bats_per_camera_day > max_bats_per_camera:
            max_bats_per_camera = max_bats_per_camera_day

        # fractions = [f for _,f in sorted(zip(angles,fractions))]
        frac_ind = 2
        axs[frac_ind].set_thetalim(-np.pi, np.pi)
        axs[frac_ind].scatter(angles, fractions, s=30)
        if axis_labels_day_ind ==day_ind:
            axs[frac_ind].set_xticks(angles)
            _ = axs[frac_ind].set_xticklabels(cameras)
            axs[frac_ind].set_title('Fraction', size='x-large')

        total_ind = 1
        axs[total_ind].set_thetalim(-np.pi, np.pi)
        axs[total_ind].scatter(angles, totals, s=30)
        if axis_labels_day_ind ==day_ind:
            axs[total_ind].set_xticks(angles)
            _ = axs[total_ind].set_xticklabels(cameras)
            axs[total_ind].set_title('Bats per degree', size='x-large')

        contrib_ind = 0
        axs[contrib_ind].set_thetalim(-np.pi, np.pi)
        axs[contrib_ind].scatter(angles, contribution, s=30)
        if axis_labels_day_ind ==day_ind:
            axs[contrib_ind].set_xticks(angles)
            _ = axs[contrib_ind].set_xticklabels(cameras)
            axs[contrib_ind].set_title('Bats from camera', size='x-large')

        pad = 60
#         left_label = f'{date} \n {int(np.sum(contribution)):,} bats'
#         axs[day_ind,0].annotate(left_label, xy=(0, 0.5), 
#                                 xytext=(-axs[day_ind,0].yaxis.labelpad - pad, 0),
#                                 xycoords=axs[day_ind,0].yaxis.label, 
#                                 textcoords='offset points',
#                                 size='xx-large', ha='right', va='center'
#                                )
    for day_ind, (date, day) in enumerate(observations.items()):
        axs[contrib_ind].set_rlim(0, max_bats_per_camera * 1.1)
        axs[total_ind].set_rlim(0, max_bats_per_degree * 1.1)

    title = f'summary-for-days-combined-center-{center}'
    fig.suptitle(title.replace('-', ' '), size=30, va='center')
    if should_save:
        fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')

In [ ]:
30.24, -12.6


In [ ]:
sorted(angles)

In [ ]:
for camera, utm in all_camera_utms.items():
    cv2.circle(big_total_mean, (int(utm[0] - area_x_origin), int(utm[1] - area_y_origin)), 5, 800000, -1)

In [ ]:
from scipy.stats import vonmises

In [ ]:
data = np.zeros(1000)
data[:250] = -.5
data[250:500] = .5
# data[:500] = 0
data[500:750] = -1.0
data[750:] = 1.0

# data[:500] = .5
# data[500:] = .3

In [ ]:

kappa, loc, scale = vonmises.fit(data, fscale=1)
rv = vonmises(kappa, loc=loc)
x = np.linspace(rv.ppf(0.01),
                rv.ppf(0.99), 100)
plt.plot(x, rv.pdf(x), 'k-', lw=2, label='frozen pdf')
plt.hist(data, density=True, histtype='stepfilled', alpha=0.2)
print(kappa)
# mean, var, skew, kurt = vonmises.stats(kappa, moments='mvsk')
# kappa = 0.1
# r = vonmises.rvs(kappa, size=10, scale = 1)
# plt.hist(r,alpha=0.2)


In [ ]:
camera_utms

In [ ]:
camera_utms = {}
for camera, obs in observations['17Nov'].items():
    camera_utms[obs['camera']] = all_camera_utms[obs['camera']]

center_utm = np.array([200800, 8606900])


In [ ]:
camera_border

In [ ]:
num_cameras = None

In [ ]:
bms = []
for obs in observations['17Nov'].values():
    obs['multiplier'] = combined_bat_multiplier(FRAME_WIDTH, WINGSPAN, obs['mean_wing'], camera_distances[obs['camera']])
    print(np.sum(obs['multiplier'] * obs['direction']))

In [ ]:
for obs in 

In [ ]:
frac_sum = 0
al = []
for days in observations.values():
    total = 0
    frac_sum = 0
    al = []
    for obs in days.values():
        angle = -camera_border[obs['camera']]['cclock_angle'] + camera_border[obs['camera']]['clock_angle']
        frac = angle / (np.pi * 2)
        frac_sum += frac
        obs_total = np.sum(obs['multiplier'] * obs['direction'])
        total += obs_total * frac
        al.append(obs_total)
    print(total, 'total weighted')
    print(frac_sum, 'frac sum')
    print(np.mean(al), 'total mean')


In [ ]:
observations['20Nov']['MusolePath']


In [ ]:
cols = 5
fig, axs = plt.subplots(cols, 2, figsize=(20,20), sharey=True)
days_sums = []
day_camera_num = []
for day in observations.values():
    days_sums.append([])
    day_camera_num.append([])
    for obs_ind, obs in enumerate(day.values()): 
        axs[obs_ind%cols, obs_ind//cols].set_title(obs['camera'])
    #     ax.plot(np.cumsum(obs['bats_raw_array']))
        axs[obs_ind%cols, obs_ind//cols].scatter(
            0, np.cumsum(obs['total_unscaled']), s=30, label=obs['date'])
        axs[obs_ind%cols, obs_ind//cols].yaxis.set_major_formatter(
            mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        axs[obs_ind%cols, obs_ind//cols].legend()
#         if len(obs['passing_bats_scaled']) > 0:
#             days_sums[-1].append(np.cumsum(obs['passing_bats_scaled'])[-1])
#             day_camera_num.append(1)

In [ ]:
for d in days_sums:
    if d:
        print(np.mean(d))

In [ ]:
for d in days_sums:
    if d:
        print(np.mean(d))

In [ ]:
def cross_angle(vec1, vec2):
    uvec1 = vec1 / np.linalg.norm(vec1)
    uvec2 = vec2 / np.linalg.norm(vec2)
    dot_product = np.cross(uvec1, uvec2)
    return np.arcsin(dot_product)

def angle(vec1, vec2):
    uvec1 = vec1 / np.linalg.norm(vec1)
    uvec2 = vec2 / np.linalg.norm(vec2)
    dot_product = np.dot(uvec1, uvec2)
    return np.arccos(dot_product)

In [ ]:
center_utm = utm.from_latlon(*center)
center_utm = [center_utm[0], center_utm[1]]

In [ ]:
x = [1, 1]
y = [-1,-.5]
print(angle(x, y) * 180 / np.pi)
print(cross_angle(y, x) * 180 / np.pi)

In [ ]:
import matplotlib.cm as cm
fig, ax = plt.subplots()

for camera, camera_utm in camera_utms.items():
    c = cm.viridis((camera_angles[camera] + np.pi)/ (2*np.pi))
    ax.scatter(*camera_utm, c=[c])
    ax.annotate(camera, camera_utm)

In [ ]:
camera_border

In [ ]:
camera_border['Sunset']

In [ ]:


processed_folder = '.../kasanka-bats/processed/deep-learning/19Nov'
track_files = sorted(glob.glob(os.path.join(processed_folder, '*/raw_tracks.npy')))
positions_files = sorted(glob.glob(os.path.join(processed_folder, '*/centers.npy')))

size_error_list = [0]
wing_error_list = [0]
with_multiplier= True

num_cameras = 1

observations = {}

for track_file, pos_file in zip(track_files[:num_cameras], positions_files[:num_cameras]):

    

    raw_track_list = np.load(track_file, allow_pickle=True)
    positions = np.load(pos_file, allow_pickle=True)
    num_frames = positions.shape[0]
    camera = track_file.split('/')[-2]
    
    obs = {'date': track_file.split('/')[-3],
           'camera': camera,
           'passing_bats_raw': np.zeros(num_frames),
           'passing_bats_scaled': np.zeros(num_frames)
          }

    tracks_list = threshold_short_tracks(raw_track_list, min_length_threshold=2)
    crossing_tracks_list = measure_crossing_bats(tracks_list, num_frames, frame_height, with_rects=with_multiplier)

    total_bats = 0

    for track in crossing_tracks_list:
        if with_multiplier:
            bat_multiplier = combined_bat_multiplier(FRAME_WIDTH, WINGSPAN, track['mean_wing'], camera_distances[camera])
        if track['crossed'] > 0:
            if with_multiplier:
                total_bats += bat_multiplier
            obs['passing_bats_raw'][track['crossed']] += 1
            if with_multiplier:
                obs['passing_bats_scaled'][track['crossed']] += bat_multiplier
        
        if track['crossed'] < 0:
            if with_multiplier:
                total_bats -= bat_multiplier 
            obs['passing_bats_raw'][-track['crossed']] -= 1
            if with_multiplier:
                obs['passing_bats_scaled'][-track['crossed']] -= bat_multiplier
    print(total_bats, camera)
    
    obs['frames'] = np.argwhere(obs['passing_bats_raw'] != 0)
    obs['passing_bats_raw'] = obs['passing_bats_raw'][obs['frames']]
    obs['passing_bats_scaled'] = obs['passing_bats_scaled'][obs['frames']]
    observations.append(obs)
    
for obs in observations:
#     plt.figure()
    fig, ax = plt.subplots(1,1)
    ax.set_title(obs['camera'])
#     ax.plot(np.cumsum(obs['bats_raw_array']))
    ax.scatter(obs['frames'], np.cumsum(obs['passing_bats_scaled']), s=1)
    ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
for obs in observations:
#     plt.figure()
    fig, ax = plt.subplots(1,1)
    ax.set_title(obs['camera'])
#     ax.plot(np.cumsum(obs['bats_raw_array']))
    ax.scatter(obs['frames'], np.cumsum(obs['passing_bats_scaled']), s=1)
    ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
plt.plot(np.cumsum(cross))
plt.figure()
plt.plot(inds, np.cumsum(vals))

In [ ]:
processed_folder = '.../Elements/bats/processed' 

track_files = sorted(glob.glob(os.path.join(processed_folder, '*/raw_tracks.npy')))
positions_files = sorted(glob.glob(os.path.join(processed_folder, '*/centers.npy')))
num_observations = None

observation_names = []
observations = {}
for track_file, pos_file in zip(track_files[0:num_observations], positions_files[0:num_observations]):
    observation_name = track_file.split('/')[-2]
    
#     if name in list(observations.keys()):
#         print(observations[name]['frames_in_prev_clips'])
        
#         tracks = np.load(track_file, allow_pickle=True)
#         print(len(tracks))
#         positions_file = os.path.join(os.path.dirname(track_file), 'positions.npy')
#         if os.path.exists(positions_file):
#             positions = np.load(positions_file, allow_pickle=True)
#         else:
#             print('{} doesn\'t have positions file. Skipping.'.format(name))
# #         for track in tracks:
# #             track['first_frame'] += observations[name]['frames_in_prev_clips']
# #             track['last_frame'] += observations[name]['frames_in_prev_clips']
            
#         observations[name]['frames_in_prev_clips'] += len(positions)
#         observations[name]['tracks'] = np.concatenate((observations[name]['tracks'], 
#                                                       tracks))
#         observations[name]['positions']= np.concatenate((observations[name]['positions'],
#                                                         positions))
#         observations[name]['files'].append(track_file)
        
        
#     else:
    tracks = np.load(track_file, allow_pickle=True)
#     positions_file = os.path.join(os.path.dirname(track_file), 'positions.npy')
#     position_files = sorted(glob.glob(os.path.join(processed_folder, observation_name, '*/positions-final.npy')))
#     positions = np.concatenate([np.load(file, allow_pickle=True) for file in position_files]) 
    positions = np.load(pos_file, allow_pickle=True)
    obs_dict = {'tracks': tracks,
                'files': [track_file], 
                'positions': positions,
                'frames_in_prev_clips': len(positions),
                'name': observation_name}
    observations[observation_name] = obs_dict
    observation_names.append(observation_name)
    print(observation_name, observations[observation_name]['positions'].shape)

In [ ]:
size_error_list = [0]
wing_error_list = [0]
with_multiplier= True


    

for camera in observations.keys():
# for camera in ['Sunset']:

    obs = observations[camera]

    raw_track_list = obs['tracks']
    num_frames = obs['positions'].shape[0]
    
    bats_raw_array = np.zeros(num_frames)
    bats_scaled_array = np.zeros(num_frames)

            
#     error = np.ones((len(size_error_list), len(wing_error_list), num_frames))
    
    frames_in_prev_clips = obs['frames_in_prev_clips']

    tracks_list = threshold_short_tracks(raw_track_list, min_length_threshold=2)
    crossing_tracks_list = measure_crossing_bats(tracks_list, num_frames, frame_height, with_rects=with_multiplier)

    total_bats = 0

    # for wing_width_pixels in forward_mean_wing:
    for track in crossing_tracks_list:
#         for size_error_ind, size_error in enumerate(size_error_list):
#     for wing_error_ind, wing_error in enumerate(wing_error_list):
#         height = calculate_height(track['mean_wing'], HCONST, WINGSPAN)
#         height += size_error
#         height = np.max(height, 0)
#         if np.isnan(height):
#             print(wing_width_pixels)
#         bat_multiplier = calculate_bat_multiplier(height, HFOV, camera_distances[camera])
        if with_multiplier:
            bat_multiplier = combined_bat_multiplier(FRAME_WIDTH, WINGSPAN, track['mean_wing'], camera_distances[camera])
        if track['crossed'] > 0:
            if with_multiplier:
                total_bats += bat_multiplier
            obs['bats_raw_array'][track['crossed']] += 1
#             obs['bats_scaled_array'][track['crossed']] += bat_multiplier
            if with_multiplier:
                obs['bats_new_scaled_array'][track['crossed']] += bat_multiplier
#             error[size_error_ind, wing_error_ind, track['crossed']] += bat_multiplier
        
        if track['crossed'] < 0:
            if with_multiplier:
                total_bats -= bat_multiplier 
            obs['bats_raw_array'][-track['crossed']] -= 1
#             obs['bats_scaled_array'][-track['crossed']] -= bat_multiplier
            if with_multiplier:
                obs['bats_new_scaled_array'][-track['crossed']] -= bat_multiplier
#             error[size_error_ind, wing_error_ind, -track['crossed']] += -bat_multiplier
    print(total_bats, camera)

In [ ]:
import matplotlib as mpl
for obs in observations.values():
#     plt.figure()
    fig, ax = plt.subplots(1,1)
    ax.set_title(obs['name'])
#     ax.plot(np.cumsum(obs['bats_raw_array']))
    ax.plot(np.cumsum(obs['bats_new_scaled_array']))
    ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
root_frame_folder = ".../Elements/bats/16Nov"
observation_name = "Sunset"

frame_files = sorted(glob.glob(os.path.join(root_frame_folder, observation_name, "*/*.jpg")))


In [ ]:
frame_ind = 20001
frame = cv2.imread(frame_files[frame_ind])
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
draw_tracks_on_frame(frame, frame_ind, 
                     observations[observation_name]['tracks'], 
                     observations[observation_name]['positions'],
                     draw_whole_track=False)

In [ ]:
print('number of tracks without min threshold' , len(raw_track_list))


    
plt.hist(track_lengths, bins=100)
plt.show()
print('number of tracks above min threshold' , len(track_lengths))

In [ ]:
# D_CHYNIANGALE = 306.0099 # meters
# D_PUKU = 383
# D_FIBWE_PARKING = 1002
# D_SUNSET = 448
# D_NOT_CHYNIANGALE = 266
# D_MUSOLE_PARKING = 408
# D_MUSOLE_PATH = 185
# D_FIBWE_PUBLIC = 1024
# D_MUSOLE_TOWER = 256


In [ ]:
print(observations.keys())


In [ ]:
observations['BBC'].keys()

In [ ]:
# r = []
# for c in track['contour']:
#     if len(c.shape) >  1:
#         rect = cv2.minAreaRect(c)
#         r.append(rect[1])
#     else:
#         r.append((np.nan, np.nan))
# r = np.array(r)    

In [ ]:
# np.array(r).shape

In [ ]:
# max_edge = np.nanmax(r, 1)
# print(max_edge)

In [ ]:
sorted(glob.glob('.../kasanka-bats/gopros/17Nov/card-g/*.MP4'))

In [ ]:
sorted(glob.glob('.../kasanka-bats/processed/17Nov/*/GH062505*'))

In [ ]:
for obs in observations.values():
    plt.figure()
    plt.title(obs['name'])
    plt.plot(np.cumsum(obs['bats_scaled_array']))
    plt.plot(np.cumsum(obs['bats_new_scaled_array']))

In [ ]:
bat_wingspan = np.array([[.819, .810, .792, .8063, np.nan],
          [.794, .817, .804, .760, .803],
          [.755, .775, .769, .754, .784],
          [.906, .882, .885, .902, .878],
          [.828, .813, .834, .842, .828]])

In [ ]:
mean_wingspan = np.nanmean(bat_wingspan, 1)


In [ ]:
bat_masses_plosone = [284,244,274,246,239,277,247,321,300,270,310,305,300,292,255,280]
bat_masses_jeb = [254, 266, 278, 326, 332]


In [ ]:
plt.scatter(bat_masses_jeb, mean_wingspan)
plt.figure()
plt.hist(bat_masses_plosone)
plt.hist(bat_masses_jeb)

In [ ]:
plt.plot(np.cumsum(bats_raw_array))
plt.figure()
for size_error_ind in range(len(size_error_list)):
    for wing_error_ind in range(len(wing_error_list)):
        plt.plot(np.cumsum(error[size_error_ind, wing_error_ind]))

In [ ]:
plt.plot(crossing_tracks_list[200]['size'])

In [ ]:
plt.plot(forward_size[0])
plt.plot(np.ones(len(forward_size[0]))*forward_mean_size[0])

track_ind = 0
while crossing_track_list[track_ind]['crossed'] < 0:
    track_ind += 1
plt.plot(np.max(crossing_track_list[track_ind]['rects'], 1))

plt.figure()

plt.hist(forward_mean_size, label='leaving bats')

plt.hist(backward_mean_size, label='coming bats')
plt.legend()

plt.figure()
plt.scatter(time_forward, forward_mean_size)
plt.scatter(time_backward, backward_mean_size)

plt.figure()
plt.scatter(time_forward, forward_mean_wing)
plt.scatter(time_backward, backward_mean_wing)

plt.figure()
plt.scatter(time_forward, forward_mean_body)

plt.figure()
plt.scatter(time_forward, np.array(forward_mean_wing)/np.array(forward_mean_body), alpha=.14)


In [ ]:
plt.plot(np.max(crossing_track_list[-19]['rects'], 1))

In [ ]:
print(calculate_bat_multiplier_simple(50, HFOV, D_CHYNIANGALE))
print(calculate_bat_multiplier(50, HFOV, D_CHYNIANGALE))

In [ ]:
simple = [calculate_bat_multiplier_simple(30, HFOV, d) for d in range(500)]
straight = [calculate_bat_multiplier(30, HFOV, d) for d in range(500)]

In [ ]:
plt.plot(simple)
plt.plot(straight)

In [ ]:
height_leaving = [calculate_height(pixels, HCONST, WINGSPAN) for pixels in forward_mean_wing]
plt.scatter(time_forward, height_leaving, alpha=.1)

plt.figure()

height_leaving = [calculate_height(pixels-1, HCONST, WINGSPAN) for pixels in forward_mean_wing]
plt.scatter(time_forward, height_leaving, alpha=.1)

plt.figure()

height_leaving = [calculate_height(pixels-4, HCONST, WINGSPAN) for pixels in forward_mean_wing]
plt.scatter(time_forward, height_leaving, alpha=.1)

In [ ]:
plt.plot(min_edge[peaks])
plt.plot(np.ones_like(min_edge[peaks]) * np.mean(min_edge[peaks]))
plt.plot(max_edge[peaks])
plt.plot(np.ones_like(max_edge[peaks]) * np.mean(max_edge[peaks]))

In [ ]:
len(height_leaving)